<a href="https://colab.research.google.com/github/dvschultz/stylegan2-ada-pytorch/blob/main/SG2_ADA_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2-ADA-PyTorch

**Notes**
* Training section should be fairly stable. I’ll slowly add features but it should work for most mainstream use cases
* Inference section is a work in progress. If you come across bug or have feature requests please post them in [Slack](https://ml-images.slack.com/archives/CLJGF384R) or on [Github](https://github.com/dvschultz/stylegan2-ada-pytorch/issues)

---

If you find this notebook useful, consider signing up for my [Patreon](https://www.patreon.com/bustbright) or [YouTube channel](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join). You can also send me a one-time payment on [Venmo](https://venmo.com/Derrick-Schultz).

## Setup

Let’s start by checking to see what GPU we’ve been assigned. Ideally we get a V100, but a P100 is fine too. Other GPUs may lead to issues.

In [ ]:
!nvidia-smi -L

Next let’s connect our Google Drive account. This is optional but highly recommended.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Install repo

The next cell will install the StyleGAN repository in Google Drive. If you have already installed it it will just move into that folder. If you don’t have Google Drive connected it will just install the necessary code in Colab.

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex

You probably don’t need to run this, but this will update your repo to the latest and greatest.

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git checkout origin/main -- train.py generate.py

## Dataset Preparation

Upload a .zip of square images to the `datasets` folder. Previously you had to convert your model to .tfrecords. That’s no longer needed :)

## Train model

Below are a series of variables you need to set to run the training. You probably won’t need to touch most of them.

* `dataset_path`: this is the path to your .zip file
* `resume_from`: if you’re starting a new dataset I recommend `'ffhq1024'` or `'./pretrained/wikiart.pkl'`
* `mirror_x` and `mirror_y`: Allow the dataset to use horizontal or vertical mirroring.

In [ ]:
#required: definitely edit these!
dataset_path = './datasets/chin-morris.zip'
resume_from = './pretrained/wikiart.pkl'
aug_strength = 0.0
train_count = 0
mirror_x = True

#broken, don't use for now :(
#mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'
snapshot_count = 4

In [ ]:
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=./results --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count

### Resume Training

Once Colab has shutdown, you’ll need to resume your training. Reset the variables above, particularly the `resume_from` and `aug_strength` settings.

1. Point `resume_from` to the last .pkl you trained (you’ll find these in the `results` folder)
2. Update `aug_strength` to match the augment value of the last pkl file. Often you’ll see this in the console, but you may need to look at the `log.txt`. Updating this makes sure training stays as stable as possible.
3. You may want to update `train_count` to keep track of your training progress.

Once all of this has been reset, run that variable cell and the training command cell after it.

## Testing/Inference

Full argument readout TK

In [ ]:
!gdown --id 16xzynLVK79E2bvYRcwPV8z1sFYJm7cI0 -O /content/jpdes-p.pkl

### Generate Images

Also known as "Inference", "Evaluation" or "Testing" the model.

`--network`: Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`: This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation`: Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 


In [ ]:
!python generate.py --outdir=out/images/ --trunc=1 --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

### Interpolations

Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at different examples of interpolation below.

#### Options

`--network`: path to your .pkl file

`--interpolation`: Walk type defines the type of interpolation you want. In some cases it can also specify whether you want the z space or the w space.

`--frames`: How many frames you want to produce. Use this to manage the length of your video.

`--trunc`: truncation value

#### Linear Interpolation

In [ ]:
!python generate.py --outdir=out/video1/ --trunc=1 --process="interpolation" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

In [ ]:
!python generate.py --outdir=out/video1-w/ --space="w" --trunc=1 --process="interpolation" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

#### Slerp Interpolation

Note: Slerp in w currently isn’t supported.

In [ ]:
!python generate.py --outdir=out/video1/ --trunc=1 --process="interpolation" --interpolation="slerp" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

#### Noise Loop

In [ ]:
!python generate.py --outdir=out/video-noiseloop-0.9d/ --trunc=0.8 --process="interpolation" --interpolation="noiseloop" --diameter=0.9 --random_seed=100 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

#### Circular Loop

In [ ]:
!python generate.py --outdir=out/video-circularloop/ --trunc=1 --process="interpolation" --interpolation="circularloop" --diameter=800.00 --frames=720 --random_seed=80 --network=/content/jpdes-p.pkl